<a href="https://colab.research.google.com/github/TheWitcher41/ER_SYS/blob/main/ImageCAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
xiaoweixumedicalai_imagecas_path = kagglehub.dataset_download('xiaoweixumedicalai/imagecas')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

dataset_path = "/kaggle/input/imagecas"
combined_zip_path = "/kaggle/working/combined.zip"
extracted_path = "/kaggle/working/extracted"

archive_parts = [
    "1-200.z01", "1-200.z02", "1-200.z03", "1-200.z04", "1-200.change2zip",
    "201-400.z01", "201-400.z02", "201-400.z03", "201-400.z04", "201-400.change2zip",
    "401-600.z01", "401-600.z02", "401-600.z03", "401-600.z04", "401-600.change2zip",
    "601-800.z01", "601-800.z02", "601-800.z03", "601-800.z04", "601-800.change2zip",
    "801-1000.z01", "801-1000.z02", "801-1000.z03", "801-1000.z04", "801-1000.change2zip"
]

print("Combining multi-part files...")
with open(combined_zip_path, "wb") as combined_zip:
    for part in archive_parts:
        part_path = os.path.join(dataset_path, part)
        if os.path.exists(part_path):
            print(f"Adding {part}...")
            with open(part_path, "rb") as part_file:
                combined_zip.write(part_file.read())
        else:
            print(f"Warning: {part} not found. Skipping...")

print("Extracting the combined ZIP file...")
os.makedirs(extracted_path, exist_ok=True)
try:
    with zipfile.ZipFile(combined_zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)
    print("Extraction complete!")
except zipfile.BadZipFile:
    print("Error: The combined file is not a valid ZIP file. Please check the multi-part files.")

print("Verifying extracted files...")
extracted_files = os.listdir(extracted_path)
print(f"Extracted {len(extracted_files)} files/folders:")
for file in extracted_files:
    print(f"- {file}")

print("Cleaning up...")
if os.path.exists(combined_zip_path):
    os.remove(combined_zip_path)
    print(f"Deleted {combined_zip_path} to free up space.")

print("Done!")

In [ ]:
extracted_files = os.listdir(extracted_path)
print("Extracted files/folders:", extracted_files)

for root, dirs, files in os.walk(extracted_path):
    print(f"Directory: {root}")
    print(f"Subdirectories: {dirs}")
    print(f"Files: {files}")
    print("-" * 40)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

image_path = os.path.join(extracted_path, "example_image.jpg")
image = Image.open(image_path)
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
csv_path = os.path.join(extracted_path, "Coronary_Segmentation_deep_learning/data_list/split_1000.csv")
data = pd.read_csv(csv_path)

print(data.head())

In [ ]:
!pip install torch torchvision opencv-python pandas scikit-image pyyaml

In [ ]:
import sys
import yaml

project_path = os.path.join(extracted_path, "Coronary_Segmentation_deep_learning")
sys.path.append(project_path)

config_path = os.path.join(project_path, "config/config.yaml")
with open(config_path, "r") as config_file:
    config = yaml.safe_load(config_file)

from model.U_Net import UNet

model = UNet(config["model"]["in_channels"], config["model"]["out_channels"])

print(model)

In [ ]:
from data.Image_loader import ImageLoader
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

image_loader = ImageLoader(config["data"]["image_dir"], config["data"]["mask_dir"])

train_loader = DataLoader(image_loader, batch_size=config["training"]["batch_size"], shuffle=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=config["training"]["learning_rate"])

for epoch in range(config["training"]["epochs"]):
    for images, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
from utils.Calculate_metrics import calculate_metrics

metrics = calculate_metrics(model, train_loader)
print(metrics)

In [ ]:
model_save_path = "/kaggle/working/unet_model.pth"
torch.save(model.state_dict(), model_save_path)

results_path = "/kaggle/working/results.csv"
metrics.to_csv(results_path, index=False)